# Global registration with RANSAC
We are going to use open3d (http://www.open3d.org/) to handle  pointclouds and generation of pointclouds

So make sure to call **pip install open3d**


In [1]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor): # recolor the points
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None): # transforma source to targets
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

We need to read in our pointclouds.

In [2]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be benefitial work on a downsampled version of the point cloud.
you can use [```pointcloudname.voxel_down_sample()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html) where pointcloud is the name of your point cloud object.

We also need to estimate the normals of the pointcloud points using [```pointcloudname.estimate_normals()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html)

And finally find fpfh features or correspondance of the downsampled point clouds.
[```o3d.registration.compute_fpfh_feature()```](http://www.open3d.org/docs/latest/python_api/open3d.registration.compute_fpfh_feature.html#open3d.registration.compute_fpfh_feature)


In [3]:
####
# Downsample and find features here
####
# Code
def get_fpfh(cp):
    cp = cp.voxel_down_sample(voxel_size)
    cp.estimate_normals()
    return cp, o3d.registration.compute_fpfh_feature(cp, o3d.geometry.KDTreeSearchParamHybrid(radius=5, max_nn=100))
r1, f1 = get_fpfh(source)
r2, f2 = get_fpfh(target)

### Ransac
We will now attempt to use ransac to do a global registration of the two poinclouds.

Using the function [```o3d.registration.registration_ransac_based_on_feature_matching```](http://www.open3d.org/docs/latest/python_api/open3d.registration.registration_ransac_based_on_feature_matching.html#open3d.registration.registration_ransac_based_on_feature_matching) from open3d


Try to find the transformation from r1 to r2.
```Python
point_to_point =  o3d.registration.TransformationEstimationPointToPoint(False)
```

When using ransac focus on the arguments below the rest are optional
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [4]:
####
# Call RANSAC here
####
point2point =  o3d.registration.TransformationEstimationPointToPoint(False)
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    r1, r2, 
    f1, f2, 
    voxel_size*1.5, 
point2point)
draw_registrations(r1, r2, ransac_result.transformation, True)
# ransac_result = ...
# draw_registrations(source, target, ransac_result.transformation, True)

## Exercises
### A)
    Can you get a decent transformation from r1 to r3?
### B)
    Try to use pruning to stop Ransac early. A pruning step takes fast pruning algorithms to quickly reject false matches early.

Open3D provides the following pruning algorithms:

- **CorrespondenceCheckerBasedOnDistance** checks if aligned point clouds are close (less than specified threshold).

- **CorrespondenceCheckerBasedOnEdgeLength** checks if the lengths of any two arbitrary edges (line formed by two vertices) individually drawn from source and target correspondences are similar. This tutorial checks that ||edge_source||>0.9×||edge_target|| and ||edge_target||> 0.9×||edge_source|| are true.

- **CorrespondenceCheckerBasedOnNormal** considers vertex normal affinity of any correspondences. It computes dot product of two normal vectors. It takes radian value for the threshold.


You can also try tweaking the voxel_size
```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

# Checkers
c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0, c1, c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point,
    checkers = checker_list)
```

### D)
Try to use **RANSACConvergenceCriteria** to see how many iterations are needed for decent convergence for both point to point and point to plane.

Replace point_to_point with point_to_plane.
```Python
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

crit = o3d.registration.RANSACConvergenceCriteria(1000000, 100)

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_plane,
    checkers = checker_list
    criteria = crit)
```

In [5]:
r3 = o3d.io.read_point_cloud("ICP/r3.pcd")
r3, f3 = get_fpfh(r3)
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    r1, r3, 
    f1, f3, 
    voxel_size*1.5, point2point)
draw_registrations(r1, r3, ransac_result.transformation, True)


In [7]:
corr_length = 0.9
distance_threshold = voxel_size * 1.5

# Checkers
c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0, c1, c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    r1, r3, 
    f1, f3, 
    distance_threshold,
    point2point,
    checkers = checker_list)
draw_registrations(r1, r3, ransac_result.transformation, True)



In [8]:
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

crit = o3d.registration.RANSACConvergenceCriteria(1000000, 100)

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    r1, r3, 
    f1, f3, 
    distance_threshold,
    point_to_plane,
    checkers = checker_list,
    criteria = crit)
draw_registrations(r1, r3, ransac_result.transformation, True)